In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

client_credentials_manager = SpotifyClientCredentials(
        "26399552a8ce4d1285397254189cac50",
        "fdacfffa2dd34dbeb127dedb459f7ea3")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
def getTrackFeatures(trackURI):
    ger_data = pd.DataFrame()
    for track in trackURI:
        audio = sp.audio_analysis(track)
        segments_data =  pd.DataFrame(audio["segments"])
        pitch = segments_data.pitches.apply(pd.Series)
        pitch.columns = ["p" + str(i) for i in range(1,13)]
        timbre = segments_data.timbre.apply(pd.Series)
        timbre.columns = ["t" + str(i) for i in range(1,13)]
        segments_data = segments_data.drop(["pitches", "timbre", "loudness_end"],
                                           axis = 1)
        segments_data = segments_data.join([pitch, timbre])
        segments_data["track_id"] = track
        ger_data = ger_data.append(segments_data)
    return ger_data

In [ ]:
countriesOfInterest = ["MX", "TR", "US"]
for year in range(2001,2020):
    for month in range(1,13):
        yrnmnth = str(year) + "-" + str(month)
        tmstamp = yrnmnth + "-15T12:00:00"
        for countryCode in countriesOfInterest:
            country = sp.featured_playlists(country = countryCode, limit = 50)
            plist = country["playlists"]["items"]
            print(countryCode, "has", len(plist), "playlists")
            plist = pd.DataFrame.from_dict(plist)
            plist.to_csv("Raw Playlist Data\\" + countryCode + "_" + yrnmnth + ".csv", index = False)
            for uri in plist["uri"]:
                currplist = sp.playlist(uri)
                trackURI = []
                for track in currplist["tracks"]["items"]:
                    try:
                        trackURI = trackURI + [track["track"]["uri"]]
                    except:
                        continue
                print("Getting track features for", uri, "in", countryCode)
                trackData = getTrackFeatures(trackURI)
                trackData.to_csv("Raw Track Data\\" + countryCode + "_" + yrnmnth + "_" + uri[17:] + ".csv", index = False)

MX has 11 playlists
Getting track features for spotify:playlist:37i9dQZF1DWZjqjZMudx9T in MX
Getting track features for spotify:playlist:37i9dQZF1DX3PsJxMtn1AP in MX
Getting track features for spotify:playlist:37i9dQZF1DX9KJJbsGEwfj in MX
Getting track features for spotify:playlist:37i9dQZF1DX05Q6G4VDGLq in MX
Getting track features for spotify:playlist:37i9dQZF1DWUmxBdWX3Pp4 in MX
Getting track features for spotify:playlist:37i9dQZF1DWY7LJpmTFjnp in MX
Getting track features for spotify:playlist:37i9dQZF1DWSXAHZTiX76o in MX
Getting track features for spotify:playlist:37i9dQZF1DXcWcRIYY41Ix in MX
Getting track features for spotify:playlist:37i9dQZF1DX01RK16gDPtA in MX
Getting track features for spotify:playlist:37i9dQZF1DX3woOVffrpln in MX
Getting track features for spotify:playlist:37i9dQZF1DX4YhSbTs17ha in MX
TR has 10 playlists
Getting track features for spotify:playlist:37i9dQZF1DX32iTMwjM5Xf in TR
Getting track features for spotify:playlist:37i9dQZF1DWUplNWv18RoG in TR
Getting tra

In [9]:
track

{'added_at': '2020-02-27T15:04:54Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
  'href': 'https://api.spotify.com/v1/users/',
  'id': '',
  'type': 'user',
  'uri': 'spotify:user:'},
 'is_local': False,
 'primary_color': None,
 'track': None,
 'video_thumbnail': None}

In [ ]:
currplist